# An Interpretable Approach to Lexical Semantic Change Detection with Lexical Substitution

Collecting data.

**Word sense induction (WSI)** is the problem of grouping occurrences of an ambiguous word according to the expressed sense of this word. Recently a new approach to this task was proposed, which generates possible substitutes for the ambiguous word in a particular context using neural language models, and then clusters sparse bag-of-words vectors built from these substitutes. Here we provide solution for the second part: clusterization of word usages using substitutes that were pregenerated based on the contexts.

Algotithm:


1.   Prepared substitutes for target words in certain contexts and their probabilities from `russe_bts-rnc` are used. Substitutes are words that could be used in the same context as the target word. Probabilities are probabilities of the particular substitution.  
2.   Substitutions are lemmatized and vectorized.
3.   BoW vectors are clustered using agglomerative clustering.
4. MaxARI is calculated.


Here all necessary modules are imported.


In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from collections import Counter
import os
from datetime import datetime
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import numpy as np
from pymorphy2 import MorphAnalyzer
from pathlib import Path
from time import time
import regex as re
from sklearn.feature_extraction import DictVectorizer
from sklearn.naive_bayes import BernoulliNB
from joblib import Memory
from sklearn.metrics import adjusted_rand_score as ARI
from sklearn.cluster import AgglomerativeClustering
# import sklearn.cluster.hierarchical
from scipy.spatial.distance import cdist
import seaborn as sns
from sklearn.metrics import silhouette_score
from tqdm.notebook import tqdm
from IPython.display import display

## Predicting substitutes

In [2]:
import torch

In [3]:
def mask_before_target(idxs, line):

    start_id = int(idxs.split(',')[0].split('-')[0].strip())
    end_id = int(idxs.split(',')[0].split('-')[1].strip())
    return line[:start_id] + '[MASK] а также ' + line[start_id:]

def mask_after_target(idxs, line):

    start_id = int(idxs.split(',')[0].split('-')[0].strip())
    end_id = int(idxs.split(',')[0].split('-')[1].strip())
    return line[:end_id+1] + ' а также [MASK]' + line[end_id+1:]

In [5]:
df = pd.read_csv('./russe-wsi-kit/data/main/bts-rnc/train.csv', sep='\t')

In [6]:
df

,context_id,word,gold_sense_id,predict_sense_id,positions,context
0,1,балка,1,NaN,90-94,"маленькой комнаты. Он был очень высок, наклони..."
1,2,балка,1,NaN,69-73,Пантюхин в Склифе сейчас. Он выползти на улицу...
2,3,балка,1,NaN,115-121,равнозначно обеспечивает и меланхоличную езду....
3,4,балка,1,NaN,85-90,"верхняя часть закрыта, замкнута, многократно о..."
4,5,балка,1,NaN,66-70,"по телевизору: наши гол забили, я вскочил от р..."
...,...,...,...,...,...,...
3486,3487,штамп,4,NaN,81-86,"дурак, но партию свою отрабатывал точно, по ус..."
3487,3488,штамп,4,NaN,85-90,"дело... получается, мыслить и выражать свое мн..."
3488,3489,штамп,4,NaN,71-77,", что актер должен иметь пять штампов-клише ""п..."
3489,3490,штамп,4,NaN,107-112,сегодняшний день в системе негосударственного ...


In [7]:
df['masked_before_target'] = df[['positions', 'context']].apply(lambda x: mask_before_target(*x), axis=1)
df['masked_after_target'] = df[['positions', 'context']].apply(lambda x: mask_after_target(*x), axis=1)

In [8]:
sent = df.iloc[2]['masked_after_target']

In [9]:
sent

'равнозначно обеспечивает и меланхоличную езду. Цельнометаллическое тело, усиленное передними и задними поперечными балками а также [MASK], наряду с работой подвески превращает машину в глыбу, всей плоскостью вросшую в'

In [10]:
from transformers import BertTokenizer, BertForMaskedLM

tokenizer = BertTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")
model = BertForMaskedLM.from_pretrained("DeepPavlov/rubert-base-cased")

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:
def predict_masked_sent(text, model, tokenizer, top_k=5):
    # Tokenize input
    text = "[CLS] %s [SEP]"%text
    tokenized_text = tokenizer.tokenize(text)
    masked_index = tokenized_text.index("[MASK]")
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    tokens_tensor = torch.tensor([indexed_tokens])
    # tokens_tensor = tokens_tensor.to('cuda')    # if you have gpu

    # Predict all tokens
    with torch.no_grad():
        outputs = model(tokens_tensor)
        predictions = outputs[0]

    probs = torch.nn.functional.softmax(predictions[0, masked_index], dim=-1)
    top_k_weights, top_k_indices = torch.topk(probs, top_k, sorted=True)

    out = []
    for i, pred_idx in enumerate(top_k_indices):
        predicted_token = tokenizer.convert_ids_to_tokens([pred_idx])[0]
        token_weight = top_k_weights[i]
        out.append((token_weight, predicted_token))

    return out

In [14]:
sent = df.iloc[0]['masked_after_target']
sent

'маленькой комнаты. Он был очень высок, наклонил голову, словно подпирая плечом потолочную балку а также [MASK], посмотрел на Сьянову серьезными черными глазами. -- Я из Москвы. Буду испытывать здесь'

In [15]:
df

,context_id,word,gold_sense_id,predict_sense_id,positions,context,masked_before_target,masked_after_target
0,1,балка,1,NaN,90-94,"маленькой комнаты. Он был очень высок, наклони...","маленькой комнаты. Он был очень высок, наклони...","маленькой комнаты. Он был очень высок, наклони..."
1,2,балка,1,NaN,69-73,Пантюхин в Склифе сейчас. Он выползти на улицу...,Пантюхин в Склифе сейчас. Он выползти на улицу...,Пантюхин в Склифе сейчас. Он выползти на улицу...
2,3,балка,1,NaN,115-121,равнозначно обеспечивает и меланхоличную езду....,равнозначно обеспечивает и меланхоличную езду....,равнозначно обеспечивает и меланхоличную езду....
3,4,балка,1,NaN,85-90,"верхняя часть закрыта, замкнута, многократно о...","верхняя часть закрыта, замкнута, многократно о...","верхняя часть закрыта, замкнута, многократно о..."
4,5,балка,1,NaN,66-70,"по телевизору: наши гол забили, я вскочил от р...","по телевизору: наши гол забили, я вскочил от р...","по телевизору: наши гол забили, я вскочил от р..."
...,...,...,...,...,...,...,...,...
3486,3487,штамп,4,NaN,81-86,"дурак, но партию свою отрабатывал точно, по ус...","дурак, но партию свою отрабатывал точно, по ус...","дурак, но партию свою отрабатывал точно, по ус..."
3487,3488,штамп,4,NaN,85-90,"дело... получается, мыслить и выражать свое мн...","дело... получается, мыслить и выражать свое мн...","дело... получается, мыслить и выражать свое мн..."
3488,3489,штамп,4,NaN,71-77,", что актер должен иметь пять штампов-клише ""п...",", что актер должен иметь пять штампов-клише ""п...",", что актер должен иметь пять штампов-клише ""п..."
3489,3490,штамп,4,NaN,107-112,сегодняшний день в системе негосударственного ...,сегодняшний день в системе негосударственного ...,сегодняшний день в системе негосударственного ...


In [16]:
from tqdm.auto import tqdm
tqdm.pandas()

In [17]:
df['masked_before_target'].iloc[1]

'Пантюхин в Склифе сейчас. Он выползти на улицу успел, а на Золоткова [MASK] а также балка обрушилась. Эх, душой компании парень был! 28-летний Геннадий так и не'

In [18]:
df['before_subst_prob'] = df['masked_before_target'].progress_apply(lambda x:
                                          predict_masked_sent(x, model, tokenizer, top_k=150))

  0%|          | 0/3491 [00:00<?, ?it/s]

In [19]:
df['after_subst_prob'] = df['masked_after_target'].progress_apply(lambda x:
                                          predict_masked_sent(x, model, tokenizer, top_k=150))

  0%|          | 0/3491 [00:00<?, ?it/s]

In [20]:
import stanza
import pymorphy2

In [21]:
nlp = stanza.Pipeline('ru', processors='tokenize,pos,lemma,depparse')
morph = pymorphy2.MorphAnalyzer()

2022-02-23 10:44:18 INFO: Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |
| pos       | syntagrus |
| lemma     | syntagrus |
| depparse  | syntagrus |

2022-02-23 10:44:18 INFO: Use device: cpu
2022-02-23 10:44:18 INFO: Loading: tokenize
2022-02-23 10:44:18 INFO: Loading: pos
2022-02-23 10:44:18 INFO: Loading: lemma
2022-02-23 10:44:19 INFO: Loading: depparse
2022-02-23 10:44:19 INFO: Done loading processors!


In [22]:
morph.parse('балкой')[0].tag

OpencorporaTag('NOUN,inan,femn sing,ablt')

In [23]:
def extract_ling_feats(idx, text):
    start_id = int(idx.split('-')[0].strip())
    end_id = int(idx.split('-')[1].strip())
    processed = nlp(text)
    case = morph.parse(text[start_id:end_id+1])[0].tag.case
    number = morph.parse(text[start_id:end_id+1])[0].tag.number
    dep=''
    for token in processed.iter_tokens():
        if start_id == token.start_char:
            dep = token.words[0].deprel
            break
    return case, number, dep

In [24]:
df[['case', 'number', 'dep']] = df[['positions', 'context']].progress_apply(lambda x:
                                          extract_ling_feats(*x), axis=1, result_type='expand')

  0%|          | 0/3491 [00:00<?, ?it/s]

In [25]:
for i in ['case', 'number', 'dep']:
    df[i + '_enc'], _ = pd.factorize(df[i.strip()])

In [27]:
from sklearn import preprocessing

x = df[['case_enc', 'number_enc', 'dep_enc']].values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df[['case_enc', 'number_enc', 'dep_enc']] = pd.DataFrame(x_scaled)

In [28]:
df.head(5)

,context_id,word,gold_sense_id,predict_sense_id,positions,context,masked_before_target,masked_after_target,before_subst_prob,after_subst_prob,case,number,dep,case_enc,number_enc,dep_enc
0,1,балка,1,NaN,90-94,"маленькой комнаты. Он был очень высок, наклони...","маленькой комнаты. Он был очень высок, наклони...","маленькой комнаты. Он был очень высок, наклони...","[(tensor(0.9854), ,), (tensor(0.0026), плиту),...","[(tensor(0.1272), пол), (tensor(0.0548), ноги)...",accs,sing,obj,0.142857,0.5,0.000000
1,2,балка,1,NaN,69-73,Пантюхин в Склифе сейчас. Он выползти на улицу...,Пантюхин в Склифе сейчас. Он выползти на улицу...,Пантюхин в Склифе сейчас. Он выползти на улицу...,"[(tensor(0.9077), ,), (tensor(0.0180), –), (te...","[(tensor(0.2998), балка), (tensor(0.0359), тож...",nomn,sing,nsubj,0.285714,0.5,0.047619
2,3,балка,1,NaN,115-121,равнозначно обеспечивает и меланхоличную езду....,равнозначно обеспечивает и меланхоличную езду....,равнозначно обеспечивает и меланхоличную езду....,"[(tensor(0.7706), ,), (tensor(0.0491), связями...","[(tensor(0.0542), балками), (tensor(0.0386), у...",ablt,plur,obl,0.428571,1.0,0.095238
3,4,балка,1,NaN,85-90,"верхняя часть закрыта, замкнута, многократно о...","верхняя часть закрыта, замкнута, многократно о...","верхняя часть закрыта, замкнута, многократно о...","[(tensor(0.9830), ,), (tensor(0.0011), (), (te...","[(tensor(0.0831), балками), (tensor(0.0274), б...",ablt,sing,advcl,0.428571,0.5,0.142857
4,5,балка,1,NaN,66-70,"по телевизору: наши гол забили, я вскочил от р...","по телевизору: наши гол забили, я вскочил от р...","по телевизору: наши гол забили, я вскочил от р...","[(tensor(0.7925), ,), (tensor(0.0098), него), ...","[(tensor(0.0372), упал), (tensor(0.0318), уеха...",accs,sing,obl,0.142857,0.5,0.095238


In [29]:
df.to_csv('bts_rnc_substs.csv', sep='\t')

KeyboardInterrupt: 

## Data collection



`substitutes_dump` – lists of substitutions with their probablities  
`data_name` – dataset with target words and examples (contexts of word usages)

In [ ]:
substitutes_dump = '/content/xlm/russe_bts-rnc/train_1-limitNone-maxexperwordNone/modelNone-beamsearchFalse/<mask><mask>-(а-также-T)-2ltr2f_topk150_fixspacesTrue.npz&/content/xlm/russe_bts-rnc/train_1-limitNone-maxexperwordNone/modelNone-beamsearchFalse/T-(а-также-<mask><mask>)-2ltr2f_topk150_fixspacesTrue.npz&'
data_name = '/content/summer-wsi/russe-wsi-kit/data/main/bts-rnc/train.csv'

Function `load_substs` collects substitutions, probabilities and examples to one main DataFrame.

In [36]:
def load_substs(substs_fname, limit=None, drop_duplicates=False, 
                data_name = None, show=True):
    '''
    If more than one set of substitutes is used, this function combines them.
    '''
    if substs_fname.endswith('&'):
        split = substs_fname.strip('&').split('&')
        if show:
            print(f'Combining:', split)
        dfinps = [load_substs_(p, limit, drop_duplicates, data_name) \
                  for p in split]
        res = dfinps[0]
        nm = len(split[0].split('<mask>'))-1
        for dfinp in dfinps[1:]:
            res = res.merge(dfinp, on=['context','positions'], \
                            how='inner', suffixes=('','_y'))
            res.substs_probs = intersect_sparse(res.substs_probs, \
                                                res.substs_probs_y, \
                                                nmasks=nm, s=0.0)        
            res.drop(columns=[c for c in res.columns if c.endswith('_y')], \
                     inplace=True)
        if show:
            print('\nExamples for one meaning:')
            ex = res.loc[(res['word'] == 'балка') & \
                          (res['gold_sense_id'] == 1)].head(2)
            print('\nExample #1\nword:', tuple(ex.word)[0],
              '\ngold_sense_id:', tuple(ex.gold_sense_id)[0],
              '\ncontext:', tuple(ex.context)[0], '\nsubstitutes & probabilities:',
              tuple(ex.substs_probs)[0])
            print('\nExample #2\nword:', tuple(ex.word)[1],
              '\ngold_sense_id:', tuple(ex.gold_sense_id)[1],
              '\ncontext:', tuple(ex.context)[1], '\nsubstitutes & probabilities:',
              tuple(ex.substs_probs)[1])


            print('\nExamples for differrent menings:')
            ex1 = res.loc[(res['word'] == 'балка') & \
                          (res['gold_sense_id'] == 1)].head(1)
            ex2 = res.loc[(res['word'] == 'балка') & \
                          (res['gold_sense_id'] == 2)].head(1)
            print('\nExample #1\nword:', tuple(ex1.word)[0],
              '\ngold_sense_id:', tuple(ex1.gold_sense_id)[0],
              '\ncontext:', tuple(ex1.context)[0], '\nsubstitutes & probabilities:',
              tuple(ex1.substs_probs)[0])
            print('\nExample #2\nword:', tuple(ex2.word)[0],
              '\ngold_sense_id:', tuple(ex2.gold_sense_id)[0],
              '\ncontext:', tuple(ex2.context)[0], '\nsubstitutes & probabilities:',
              tuple(ex2.substs_probs)[0])
        return res
    elif substs_fname.endswith('+'): 
        split = substs_fname.strip('+').split('+') # комментарий
        p1 = '+'.join(split[:-1])
        s = float(split[-1]) 
        p2 = re.sub(r'((<mask>)+)(.*?)T',r'T\3\1',p1)
        if p2==p1:
            p2 =  re.sub(r'T(.*?)((<mask>)+)',r'\2\1T',p1)
        print(f'Combining {p1} and {p2}')
        if p1==p2:
            raise Exception('Cannot conver fname to symmetric one:', p1)
        dfinp1, dfinp2 = (load_substs_(p, limit, drop_duplicates, data_name) \
                          for p in (p1,p2))
        dfinp = dfinp1.merge(dfinp2, on=['context','positions'], how='inner', \
                             suffixes=('','_y'))
        dfinp.substs_probs = intersect_sparse(dfinp.substs_probs, \
                                              dfinp.substs_probs_y, \
                                              nmasks=len(substs_fname.split('<mask>'))-1, \
                                              s=s)
        dfinp.drop(columns=[c for c in dfinp.columns if c.endswith('_y')], \
                   inplace=True)
        return dfinp
    else:
        return load_substs_(substs_fname, limit, drop_duplicates, data_name)

In [37]:
def load_substs_(substs_fname, limit=None, drop_duplicates=True, data_name = None):
    '''
    Collects substitutions, probabilities and examples to one main DataFrame.
    '''
    st = time()
    p = Path(substs_fname) 
    npz_filename_to_save = None
    # print(time()-st, 'Loading substs from ', p)

    # substitutions and probabilities
    if substs_fname.endswith('.npz'): 
        arr_dict = np.load(substs_fname, allow_pickle=True)
        # separate dictionaries for substitutions and probabilities
        ss,pp = arr_dict['substs'], arr_dict['probs'] 
        ss,pp = [list(s) for s in ss], [list(p) for p in pp]
        # creating a DataFrame
        substs_probs = pd.DataFrame({'substs':ss, 'probs':pp}) 
        substs_probs = substs_probs.apply(lambda r: [(p,s) for s,p in zip(r.substs, r.probs)], axis=1) 

    # examword usages and contexts
    p_ex = p.parent / (p.name+'.input')
    if os.path.isfile(p_ex):
        # print(time()-st,'Loading examples from ', p_ex)
        dfinp = pd.read_csv(p_ex, nrows=limit)
        dfinp['positions'] = dfinp['positions'].apply(pd.eval).apply(tuple)
        dfinp['word_at'] = dfinp.apply(lambda r: r.context[slice(*r.positions)], 
                                       axis=1)  # word_at stores wordform as it occured in text

    dfinp.positions = dfinp.positions.apply(tuple)
    # adding substitutions and probabilities to main DF
    dfinp['substs_probs'] = substs_probs 
    if drop_duplicates: # deleting duplicates
        dfinp = dfinp.drop_duplicates('context')
    dfinp.reset_index(inplace = True)
    # display(dfinp)
    dfinp['positions'] = dfinp.positions.apply(tuple)
    return dfinp

Here we choose best substitutes for target word using substitutes for both templates by multiplication of probabilities. 

\begin{align}
\mathcal{P}^{final} = P(s_i|Context, templ_1) \cdot P(s_i|Context, templ_2)
\end{align} 

But we do not want to throw immediately the substitute if it is in only one set (generated for one template), so we use product of smoothed probability distributions as a final proba.  

---
Here we find smoothing rates.

$\alpha_{1,2}$ is to represent the proba of any substitution not inclided in the substitutes candidate set. This evenly (across all the possible $250000$ tokens) distributes the remainder calcilated as $1 - \text{sum of all the known substitutes probas}$.

\begin{align}
\alpha_1 = \frac{1 - \sum_{i=1}^{n_{substs\_probs_1}}P_{1i}}{ 250000^{n_{masks}}}
\end{align} 

\begin{align}
\alpha_2 = \frac{1 - \sum_{i=1}^{n_{substs\_probs_2}}P_{2i}}{ 250000^{n_{masks}}}
\end{align} 

---

Then we find a smoothed probability for each substitute provided by context.

\begin{align}
P(s_i|Context, templ_1) = 
    \left\{
\begin{array}{cl}
P_{1i} + \alpha_1 & s_i \in substs\_probs_1 \\
\alpha_1 & s_i \notin substs\_probs_1
\end{array}
\right.
\end{align}  

\begin{align}
P(s_i|Context, templ_2) = 
    \left\{
\begin{array}{cl}
P_{2i} + \alpha_2 & s_i \in substs\_probs_2 \\
\alpha_2 & s_i \notin substs\_probs_2
\end{array}
\right.
\end{align}

Product of smoothed probability distributions:

\begin{align}
\mathcal{P}^{final}_{i} = 
    \left\{
\begin{array}{cl}
(P_{1i} + \alpha_1) \cdot (P2i + alpha2) & s_i \in substs\_probs_1 \land s_i \in substs\_probs_2 \\
(P_{1i} + \alpha_1) \cdot alpha2 & s_i \in substs\_probs_1 \land s_i \notin substs\_probs_2 \\
\alpha_1 \cdot (P_{2i} + \alpha_2) & s_i \notin substs\_probs_1 \land s_i \in substs\_probs_2 \\
\alpha_1 \cdot \alpha_2 & s_i \notin substs\_probs_1 \land s_i \notin substs\_probs_2
\end{array}
\right.
\end{align} 

We get two matricies (for two templates) with a probability for the substitute if it is in the certain set or zero if it is not.

\begin{align}
f_{1i} = 
    \left\{
\begin{array}{cl}
P_{1i} & s_i \in substs\_probs_1 \\
0 & s_i \notin substs\_probs_1
\end{array}
\right.
\end{align} 



\begin{align}
f_{2i} = 
    \left\{
\begin{array}{cl}
P_{2i} & s_i \in substs\_probs_2 \\
0 & s_i \notin substs\_probs_2
\end{array}
\right.
\end{align} 

Multiplication of the respective matricies ($\forall i f_{1i} \in F_1$) using product of smoothed probability distributions.

\begin{align}
\mathcal{P}^{final} = F_1 \cdot F_2 + \alpha_1 \cdot F_2 + \alpha_2 \cdot F_1 + \alpha_1 \cdot \alpha_2
\end{align} 


We do not use $\alpha_1 \cdot \alpha_2$ **in code**, because probabilities for the substitutes that are not in both sets are too low, and we need to save memory resources.

Owing to the size of the final matrix we use the sparse matrix.  
Then we find substitutes with the highest probabilities.

In [38]:
def intersect_sparse(substs_probs, substs_probs_y, nmasks=1, s=0, debug=False):
    '''
    Combines different sets of substitutes (for different tamplates) using
    product of smoothed probability distributions.
    '''
    
    vec = DictVectorizer(sparse=True)
    f1=substs_probs.apply(lambda l: {s:p for p,s in l})
    f2=substs_probs_y.apply(lambda l: {s:p for p,s in l})
    vec.fit(list(f1)+list(f2))
    f1,f2 = (vec.transform(list(f)) for f in (f1,f2)) # sparse matrix

    alpha1, alpha2 = ((1. - f.sum(axis=-1).reshape(-1,1)) / 250000**nmasks \
                      for f in (f1, f2))
    prod = f1.multiply(f2) + f1.multiply(alpha2) + f2.multiply(alpha1)
    # + alpha1*alpha2 is ignored to preserve sparsity 
    # finally, we don't want substs with 0 
    # probs before smoothing in both distribs
    fn = np.array(vec.feature_names_)
    maxlen=(substs_probs_y.apply(len)+substs_probs.apply(len)).max()
    m = prod
    n_texts = m.shape[0]
    
    def reverse_argsort(mdata):
        return np.argsort(mdata)[::-1]
    
    idx = list()
    for text_ix in range(n_texts):
      # sparce matrices are used to preserve high performance
      # refer to https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html
      # to learn the sparse matrices indexing (i.e. what is `indptr` and `data`)
        text_sparse_indices = m.indices[m.indptr[text_ix]:m.indptr[text_ix+1]]
        text_sparse_data = m.data[m.indptr[text_ix]:m.indptr[text_ix+1]]
        text_sp_data_revsorted_ixes = reverse_argsort(text_sparse_data)
        smth = text_sparse_indices[text_sp_data_revsorted_ixes]
        idx.append(smth)

    l = list()
    for text_ix, text_sparse_ixes_sorted in enumerate(idx):
        probas = m[text_ix].toarray()[0,text_sparse_ixes_sorted]
        substs = fn[text_sparse_ixes_sorted]
        good_substs = list()
        for proba, subst in zip(probas, substs):
      
            if subst.startswith(' ') and ' ' not in subst.strip():
                good_substs.append((proba, subst))
            l.append(good_substs)
 
    print('Combination: ', l[0][:10])
    return l


---

In [43]:
df

,context_id,word,gold_sense_id,predict_sense_id,positions,context,masked_before_target,masked_after_target,before_subst_prob,after_subst_prob
0,1,балка,1,NaN,90-94,"маленькой комнаты. Он был очень высок, наклони...","маленькой комнаты. Он был очень высок, наклони...","маленькой комнаты. Он был очень высок, наклони...","[(tensor(0.9854), ,), (tensor(0.0026), плиту),...","[(tensor(0.1272), пол), (tensor(0.0548), ноги)..."
1,2,балка,1,NaN,69-73,Пантюхин в Склифе сейчас. Он выползти на улицу...,Пантюхин в Склифе сейчас. Он выползти на улицу...,Пантюхин в Склифе сейчас. Он выползти на улицу...,"[(tensor(0.9077), ,), (tensor(0.0180), –), (te...","[(tensor(0.2998), балка), (tensor(0.0359), тож..."
2,3,балка,1,NaN,115-121,равнозначно обеспечивает и меланхоличную езду....,равнозначно обеспечивает и меланхоличную езду....,равнозначно обеспечивает и меланхоличную езду....,"[(tensor(0.7706), ,), (tensor(0.0491), связями...","[(tensor(0.0542), балками), (tensor(0.0386), у..."
3,4,балка,1,NaN,85-90,"верхняя часть закрыта, замкнута, многократно о...","верхняя часть закрыта, замкнута, многократно о...","верхняя часть закрыта, замкнута, многократно о...","[(tensor(0.9830), ,), (tensor(0.0011), (), (te...","[(tensor(0.0831), балками), (tensor(0.0274), б..."
4,5,балка,1,NaN,66-70,"по телевизору: наши гол забили, я вскочил от р...","по телевизору: наши гол забили, я вскочил от р...","по телевизору: наши гол забили, я вскочил от р...","[(tensor(0.7925), ,), (tensor(0.0098), него), ...","[(tensor(0.0372), упал), (tensor(0.0318), уеха..."
...,...,...,...,...,...,...,...,...,...,...
3486,3487,штамп,4,NaN,81-86,"дурак, но партию свою отрабатывал точно, по ус...","дурак, но партию свою отрабатывал точно, по ус...","дурак, но партию свою отрабатывал точно, по ус...","[(tensor(0.9119), ,), (tensor(0.0089), партии)...","[(tensor(0.6548), партии), (tensor(0.0712), па..."
3487,3488,штамп,4,NaN,85-90,"дело... получается, мыслить и выражать свое мн...","дело... получается, мыслить и выражать свое мн...","дело... получается, мыслить и выражать свое мн...","[(tensor(0.5066), -), (tensor(0.3153), ,), (te...","[(tensor(0.0991), говорит), (tensor(0.0546), н..."
3488,3489,штамп,4,NaN,71-77,", что актер должен иметь пять штампов-клише ""п...",", что актер должен иметь пять штампов-клише ""п...",", что актер должен иметь пять штампов-клише ""п...","[(tensor(0.9507), ,), (tensor(0.0273), -), (te...","[(tensor(0.3694), клише), (tensor(0.0674), сло..."
3489,3490,штамп,4,NaN,107-112,сегодняшний день в системе негосударственного ...,сегодняшний день в системе негосударственного ...,сегодняшний день в системе негосударственного ...,"[(tensor(0.8356), ,), (tensor(0.0189), :), (te...","[(tensor(0.0290), схема), (tensor(0.0248), схе..."


As you can see below, combination of substitutes improves their quality and they fit context better.

Here we see that substitutes for one meaning are similar and for different meanings are more different.


In [ ]:
df = load_substs(substitutes_dump, data_name=data_name)

Combining: ['/content/xlm/russe_bts-rnc/train_1-limitNone-maxexperwordNone/modelNone-beamsearchFalse/<mask><mask>-(а-также-T)-2ltr2f_topk150_fixspacesTrue.npz', '/content/xlm/russe_bts-rnc/train_1-limitNone-maxexperwordNone/modelNone-beamsearchFalse/T-(а-также-<mask><mask>)-2ltr2f_topk150_fixspacesTrue.npz']
Combination:  [(0.0003074132201598033, ' решетку'), (4.1475361323760285e-05, ' поверхность'), (3.976416282017758e-05, ' опору'), (2.2270818190058624e-05, ' трубу'), (2.102492416240878e-05, ' мебель'), (9.649139098880059e-06, ' спинку'), (9.437091495716981e-06, ' балку'), (8.321624868184736e-06, ' доску'), (8.03497520660322e-06, ' раму'), (7.317509182332922e-06, ' панель')]

Examples for one meaning:

Example #1
word: балка 
gold_sense_id: 1 
context: маленькой комнаты. Он был очень высок, наклонил голову, словно подпирая плечом потолочную балку, посмотрел на Сьянову серьезными черными глазами. -- Я из Москвы. Буду испытывать здесь 
substitutes & probabilities: [(0.00030741322015980

---

In [ ]:
# we will further work with this dataframe 
# (especially w the columns word, substs_probs, gold_sense_id)
df.head(3)

,index,context_id,word,gold_sense_id,predict_sense_id,positions,context,word_at,substs_probs
0,0,1,балка,1,NaN,"(90, 95)","маленькой комнаты. Он был очень высок, наклони...",балку,"[(0.0003074132201598033, решетку), (4.1475361..."
1,1,2,балка,1,NaN,"(69, 74)",Пантюхин в Склифе сейчас. Он выползти на улицу...,балка,"[(0.020003307469640956, крыша), (0.0001694636..."
2,2,3,балка,1,NaN,"(115, 122)",равнозначно обеспечивает и меланхоличную езду....,балками,"[(0.01107819609150758, колесами), (0.00561495..."
3,3,4,балка,1,NaN,"(85, 91)","верхняя часть закрыта, замкнута, многократно о...",балкой,"[(0.0018431491390275595, крышей), (0.00144177..."
4,4,5,балка,1,NaN,"(66, 71)","по телевизору: наши гол забили, я вскочил от р...",балку,"[(0.0040539591647571534, стену), (0.000310801..."
...,...,...,...,...,...,...,...,...,...
3486,3486,3487,штамп,4,NaN,"(81, 87)","дурак, но партию свою отрабатывал точно, по ус...",штампы,"[(0.0031943255190048274, партия), (9.88932979..."
3487,3487,3488,штамп,4,NaN,"(85, 91)","дело... получается, мыслить и выражать свое мн...",штампу,"[(1.0800298370078835e-05, логике), (9.5142067..."
3488,3488,3489,штамп,4,NaN,"(71, 78)",", что актер должен иметь пять штампов-клише ""п...",штампов,"[(0.0005675281293156448, линий), (0.000390070..."
3489,3489,3490,штамп,4,NaN,"(107, 113)",сегодняшний день в системе негосударственного ...,штампы,"[(0.01981654824764262, следы), (0.00222401351..."


## Preprocessing

This objects are used for morphological analysis.

In [30]:
_ma = MorphAnalyzer()
_ma_cache = {}

Function `ma(s)` gets a string with one token, deletes spaces before and after token and returns grammatical information about it. If it was met before, we would get information from the special dictionary `_ma_cache`; if it was not, information would be gotten from `pymorphy2`.

In [31]:
# ma stands for morphological analysis
def ma(s):
    '''
    Gets a string with one token, deletes spaces before and 
    after token and returns grammatical information about it. If it was met 
    before, we would get information from the special dictionary _ma_cache; 
    if it was not, information would be gotten from pymorphy2.
    '''
    s = s.strip()  # get rid of spaces before and after token, 
                   # pytmorphy2 doesn't work with them correctly
    if s not in _ma_cache:
        _ma_cache[s] = _ma.parse(s)
    return _ma_cache[s]

In [32]:
print('word:', df.loc[0, 'before_subst_prob'][0][1])

word: ,


In [33]:
print('morphological analysis:', ma(df.loc[0, 'before_subst_prob'][0][1]))

morphological analysis: [Parse(word=',', tag=OpencorporaTag('PNCT'), normal_form=',', score=1.0, methods_stack=((PunctuationAnalyzer(score=0.9), ','),))]


Function `get_nf_cnt(substs_probs)` gets substitutes and returns normal forms of substitutes and count of substitutes that coresponds to each normal form. 

In [34]:
def get_nf_cnt(substs_probs):
    '''
    Gets substitutes and returns normal 
    forms of substitutes and count of substitutes that coresponds to 
    each normal form.
    '''
    nf_cnt = Counter(nf for l in substs_probs \
                     for p, s in l for nf in {h.normal_form for h in ma(s)})
    print('\n'.join('%s: %d' % p for p in nf_cnt.most_common(10)))
    return nf_cnt

In [35]:
nf_cnt = get_nf_cnt(df['before_subst_prob'])

он: 4290
она: 4087
весь: 4039
они: 3924
и: 3648
оно: 3613
тот: 3509
,: 3491
.: 3436
-: 3407


Function `get_normal_forms(s, nf_cnt=None)` gets string with one token and returns set of most possible lemmas, all lemmas or one possible lemma.

In [36]:
def get_normal_forms(s, nf_cnt=None):
    '''
    Gets string with one token and returns set of most possible lemmas, 
    all lemmas or one possible lemma.
    '''
    hh = ma(s)
    if nf_cnt is not None and len(hh) > 1:  # select most common normal form
        h_weights = [nf_cnt[h.normal_form] for h in hh]
        max_weight = max(h_weights)
        return {h.normal_form for i, h in enumerate(hh) \
                if h_weights[i] == max_weight}
    else:
        return {h.normal_form for h in hh}

For each substitute we choose the most common lemma among all substitutes' lemmas.

In [37]:
get_normal_forms('жала')

{'жало', 'жать'}

In [38]:
print('counts:', nf_cnt['жало'], nf_cnt['жать'])

counts: 3 3


In [39]:
get_normal_forms('жала', nf_cnt)

{'жало', 'жать'}

Function `preprocess_substs(r, lemmatize=True, nf_cnt=None, exclude_lemmas={}, )` is for preprocessing of substitutions. It gets Series of substitutions and probabilities, exclude lemmas from `exclude_lemmas` if it is not empty and lemmatize them if it is needed.

In [40]:
def preprocess_substs(r, lemmatize=True, nf_cnt=None, exclude_lemmas={}):
    '''
    For preprocessing of substitutions. It gets Series of substitutions 
    and probabilities, exclude lemmas from exclude_lemmas 
    if it is not empty and lemmatize them if it is needed.
    '''
    res = [s.strip() for p, s in r]
    if exclude_lemmas:
        res1 = [s for s in res 
                if not set(get_normal_forms(s)).intersection(exclude_lemmas)]
        res = res1
    if lemmatize:
        res = [nf for s in res for nf in get_normal_forms(s, nf_cnt)]
    return res

Here substitutes are preprocessed for clusterization.

In [41]:
topk = 128 # size of the substitutes' top to use

In [42]:
substs_texts = df.apply(lambda r: preprocess_substs(r.before_subst_prob[:topk], 
                                                    nf_cnt=nf_cnt, 
                                                    lemmatize=True), 
                        axis=1).str.join(' ')

In [43]:
tokens_5 = [a[1] for a in df.loc[0, 'before_subst_prob']]
print(' '.join(tokens_5))

, плиту часть . плоскость ; стену плиты и - потолок плит башню яму панель – — стенку ( доску группу балк конструкцию стены плита платформу плоскости перекрытие балки поверхность лестницу ту зону крышу палубу шахту : живопись ее клетку ткань … / пол опору секцию технику пару на балок комнату ##ку стене перекрытия машину стен систему части ' композицию сферу галерею площадку " ярус мебель такую опоры кору ячейку кучу ступень а точку станцию кость консоль нее поверхности по посуду девушку камеру другую или кладку нижнюю постройку ##лю ) установку структуру стена ##у решетку свода строение землю плитами корзину роспись одежду тумб в этаж балка что колонку балками палату работу верев ##ую траншею цоколь раму каркас её ось стойку линию вторую долю трубу долину одну ##ь плитой третью прочность не карниз дно ведущую травму банку глубину консоли половину толщину бумагу шахты с колонну ферму металлическую ##ную под погоду перекрытий


In [44]:
print('word:', df.loc[0, 'word'], 
      '\nlemmas of substitutes:', substs_texts[0])

word: балка 
lemmas of substitutes: , плита часть . плоскость ; стена плита и - потолок плита башня яма ям панель – — стенка ( доска группа балк конструкция стена плита платформа плоскость перекрытие балка поверхность лестница тот зона крыша палуба шахта : живопись она клетка ткань … / пол опора секция техника пара на балка комната ##ка стена перекрытие машина стена система часть ' композиция сфера галерея площадка " ярус мебель такой опора кора ячейка куча ступень а точка станция кость консоль она поверхность по посуда девушка камера другой или кладка нижний постройка ##ить ) установка структура стена ##у решётка свод строение земля плита корзина роспись одежда тумба в этаж балка что колонок колонка балка палата работа верёва верёв ##ий траншея цоколь рам рама каркас она ось стойка линия второй доля труба долина один ##ь плита


`substs_texts` is the set with strings of lemmatized substitutes for each example.

In [45]:
substs_texts

0       , плита часть . плоскость ; стена плита и - по...
1       , – - — : ( ; и . … село упасть ! весь вот да ...
2       , связь узел продольный поперечный балка ( — и...
3       , ( ; — . часть и поперечный она - продольный ...
4       , он дерево машина она куча стена яма ям . рек...
                              ...                        
3486    , партия ; они … . — он - весь : они этот этот...
3487    - , – русский письмо список — текст латынь сти...
3488    , - ; . – : " — и ) ( … штука год ! а или + пл...
3489    , : проблема ( " - ) и . недостаток штамп ; « ...
3490    , элемент клише жанр стиль эпизод драма комеди...
Length: 3491, dtype: object

## Clusterization and ARI calculating

Paramters which are used for vectorization of substitutes and clusterization.

In [46]:
vectorizer = 'TfidfVectorizer'
lemmatize = True
analyzer = 'word'
min_df = 0.05
max_df = 0.95
ngram_range = (1, 1)
ncs=(2,10)

In [47]:
vec = eval(vectorizer)(token_pattern=r"(?u)\b\w+\b", 
                       min_df=min_df, max_df=max_df, 
                       analyzer=analyzer, ngram_range=ngram_range)

In [48]:
# the list of words of interest. 
# note that all the dataset entries refer to one of these words
df.word.unique()

array(['балка', 'вид', 'винт', 'горн', 'губа', 'жаба', 'клетка', 'крыло',
       'купюра', 'курица', 'лавка', 'лайка', 'лев', 'лира', 'мина',
       'мишень', 'обед', 'оклад', 'опушка', 'полис', 'пост', 'поток',
       'проказа', 'пропасть', 'проспект', 'пытка', 'рысь', 'среда',
       'хвост', 'штамп'], dtype=object)

Function `max_ari` gets data and substitutions (and some parameters like `vectorizer` and parameters for clusterization). Here for each unique word substitutions are vectorized and senses are clusterized. 

In [49]:
df[['case_enc', 'number_enc', 'dep_enc']].to_numpy()

array([[0.14285714, 0.5       , 0.        ],
       [0.28571429, 0.5       , 0.04761905],
       [0.42857143, 1.        , 0.0952381 ],
       ...,
       [0.57142857, 1.        , 0.23809524],
       [0.14285714, 1.        , 0.04761905],
       [0.14285714, 1.        , 0.        ]])

In [50]:
def max_ari(df, X, ncs,
            affinity='cosine', linkage='average', vectorizer=None):
    '''
    Gets data and substitutions (and some parameters 
    like vectorizer and parameters for clusterization). 
    Here for each unique word substitutions 
    are vectorized and senses are clusterized.
    It returns metrics of clusterization.
    '''
    sdfs = []
    for word in df.word.unique():
        # collecting examples for the word
        mask = (df.word == word)
#         print(mask)
        # vectors for substitutions
        vectors = X[mask] if vectorizer is None \
        else vectorizer.fit_transform(X[mask]).toarray()
        vectors_ling = np.hstack((vectors, df[mask][['case_enc', 'number_enc', 'dep_enc']].to_numpy()))
        # ids of senses of the examples
        gold_sense_ids = df.gold_sense_id[mask]
        gold_sense_ids = None if gold_sense_ids.isnull().any() \
        else gold_sense_ids

        # clusterization (ari is kept in sdf along with other info)
        best_clids, sdf, _ = clusterize_search(word, vectors_ling, gold_sense_ids, 
                ncs=ncs,
                affinity=affinity, linkage=linkage)
        df.loc[mask, 'predict_sense_id'] = best_clids # result ids of clusters
        sdfs.append(sdf)
    
    return sdfs

Function `clusterize_search` gets word, vectors, gold_sense_ids and provides AgglomerativeClustering of vectors of substitutes for each example of one word.


Than it calculates [ARI](https://en.wikipedia.org/wiki/Rand_index) and [silhouette scores](https://en.wikipedia.org/wiki/Silhouette_(clustering)).



In [51]:
def clusterize_search(word, vecs, gold_sense_ids = None, 
                      ncs=list(range(1, 5, 1)) + list(range(5, 12, 2)),
            affinity='cosine', linkage='average', print_topf=None,
            generate_pictures_df = False,  corpora_ids = None):
    '''
    Gets word, vectors, gold_sense_ids and provides AgglomerativeClustering.
    '''
  
    sdfs = []
    tmp_dfs = []

    # adding 1 to zero vectors, because there will be problems (all-zero vectorized entries), if they remain zero
    # this introduces a new dimension with possible values 
    # 1 -- all the other coords are zeros
    # 0 -- other coords are not all-zeros
    zero_vecs = ((vecs ** 2).sum(axis=-1) == 0)
    if zero_vecs.sum() > 0:
        vecs = np.concatenate((vecs, 
                               zero_vecs[:, np.newaxis].astype(vecs.dtype)), 
                              axis=-1)

    best_clids = None
    best_silhouette = 0
    distances = []

    # matrix with computed distances between each pair of the two collections of inputs
    distance_matrix = cdist(vecs, vecs, metric=affinity)
    distances.append(distance_matrix)

    for nc in ncs:
        # clusterization
        clr = AgglomerativeClustering(affinity='precomputed', 
                                      linkage=linkage, 
                                      n_clusters=nc)
        clids = clr.fit_predict(distance_matrix) if nc > 1 \
        else np.zeros(len(vecs))

        # computing metrics
        ari = ARI(gold_sense_ids, clids) if gold_sense_ids is not None else np.nan
        sil_cosine = -1. if len(np.unique(clids)) < 2 else silhouette_score(vecs, clids, metric='cosine')
        sil_euclidean = -1. if len(np.unique(clids)) < 2 else silhouette_score(vecs, clids, metric='euclidean')
        
        # vc like 5/4/3 says that 
        # there are 5 examples w golden_id=id1; 
        # there are 4 examples w golden_id=id2; 
        # there are 3 examples w golden_id=id3; 
        # e.g. вид 4/3/2 means that there were 
        # 4 examples with вид==view; 3 examples .==type; 2 examples .==specie
        vc = '' if gold_sense_ids is None else '/'.join(
            np.sort(pd.value_counts(gold_sense_ids).values)[::-1].astype(str))
        
        if sil_cosine > best_silhouette:
            best_silhouette = sil_cosine
            best_clids = clids

        # metrics for each word
        sdf = pd.DataFrame({'ari': ari,
                            'word': word, 'nc': nc,
                            'sil_cosine': sil_cosine,
                            'sil_euclidean': sil_euclidean,
                            'vc': vc,
                            'affinity': affinity, 'linkage': linkage}, \
                           index=[0])

        sdfs.append(sdf)
  
    sdf = pd.concat(sdfs, ignore_index=True)

    return best_clids, sdf, distances


In [52]:
sdfs = max_ari(df, 
               substs_texts, 
               ncs=range(*ncs), 
               affinity='cosine', 
               linkage='average', 
               vectorizer=vec)

Metrics for one unique word.

In [53]:
sdfs[0]

,ari,word,nc,sil_cosine,sil_euclidean,vc,affinity,linkage
0,-0.031687,балка,2,0.166024,0.026075,81/38,cosine,average
1,0.001893,балка,3,0.146471,0.015558,81/38,cosine,average
2,-0.005198,балка,4,0.142668,0.016125,81/38,cosine,average
3,0.010129,балка,5,0.127995,0.010891,81/38,cosine,average
4,-0.030015,балка,6,0.130211,0.016721,81/38,cosine,average
5,-0.029457,балка,7,0.116823,0.007606,81/38,cosine,average
6,-0.029640,балка,8,0.114049,0.005748,81/38,cosine,average
7,-0.032733,балка,9,0.108765,0.004823,81/38,cosine,average


In [54]:
def metrics(sdfs):
    # all metrics for each unique word
    sdf = pd.concat(sdfs, ignore_index=True)
    # groupby is docuented to preserve inside group order
    res = sdf.sort_values(by='ari').groupby(by='word').last()
    # maxari for fixed hypers
    fixed_hypers = sdf.groupby(['affinity', 
                                'linkage', 
                                'nc']).agg({'ari': np.mean}).reset_index()
    idxmax = fixed_hypers.ari.idxmax()
    res_df = fixed_hypers.loc[idxmax:idxmax].copy()
    res_df = res_df.rename(columns=lambda c: 'fh_maxari' if c == 'ari' \
                           else 'fh_' + c)
    res_df['maxari'] = res.ari.mean()

    for metric in [c for c in sdf.columns if c.startswith('sil')]:
        res_df[metric+'_ari'] = sdf.sort_values(by=metric).groupby(by='word').last().ari.mean()

    return res_df, res, sdf

In [55]:
res_df, res, sdf = metrics(sdfs)

## Results

General metrics for the configuration.

In [56]:
res_df

,fh_affinity,fh_linkage,fh_nc,fh_maxari,maxari,sil_cosine_ari,sil_euclidean_ari
1,cosine,average,3,0.156143,0.274359,0.148206,0.132526


Metrics for each word.

In [57]:
res

,ari,nc,sil_cosine,sil_euclidean,vc,affinity,linkage
word,,,,,,,
балка,0.010129,5,0.127995,0.010891,81/38,cosine,average
вид,0.000641,5,0.123047,0.012779,38/36/3,cosine,average
винт,0.205065,8,0.111291,0.002074,67/39/16/1,cosine,average
горн,-0.024567,2,0.114606,0.013441,30/20/1,cosine,average
губа,0.558723,2,0.284611,0.144102,132/3/2,cosine,average
жаба,0.286801,6,0.129432,0.035217,79/27/9/6,cosine,average
клетка,0.222721,7,0.161020,0.016573,96/38/7/4/4/1,cosine,average
крыло,0.427808,9,0.165801,0.043732,51/19/7/5/4/3/1/1,cosine,average
купюра,0.095901,3,0.237494,0.054303,138/12,cosine,average


In [125]:
from conllu import parse

data = """
# text = The quick brown fox jumps over the lazy dog.
1   The     the    DET    DT   Definite=Def|PronType=Art   4   det     _   _
2   quick   quick  ADJ    JJ   Degree=Pos                  4   amod    _   _
3   brown   brown  ADJ    JJ   Degree=Pos                  4   amod    _   _
4   fox     fox    NOUN   NN   Number=Sing                 5   nsubj   _   _
5   jumps   jump   VERB   VBZ  Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin   0   root    _   _
6   over    over   ADP    IN   _                           9   case    _   _
7   the     the    DET    DT   Definite=Def|PronType=Art   9   det     _   _
8   lazy    lazy   ADJ    JJ   Degree=Pos                  9   amod    _   _
9   dog     dog    NOUN   NN   Number=Sing                 5   nmod    _   SpaceAfter=No
10  .       .      PUNCT  .    _                           5   punct   _   _

# text = The quick brown fox jumps over the lazy dog.
1   The     the    DET    DT   Definite=Def|PronType=Art   4   det     _   _
2   quick   quick  ADJ    JJ   Degree=Pos                  4   amod    _   _
3   brown   brown  ADJ    JJ   Degree=Pos                  4   amod    _   _
4   fox     fox    NOUN   NN   Number=Sing                 5   nsubj   _   _
5   jumps   jump   VERB   VBZ  Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin   0   root    _   _
6   over    over   ADP    IN   _                           9   case    _   _
7   the     the    DET    DT   Definite=Def|PronType=Art   9   det     _   _
8   lazy    lazy   ADJ    JJ   Degree=Pos                  9   amod    _   _
9   dog     dog    NOUN   NN   Number=Sing                 5   nmod    _   SpaceAfter=No
10  .       .      PUNCT  .    _                           5   punct   _   _

# text = The quick brown fox jumps over the lazy dog.
1   The     the    DET    DT   Definite=Def|PronType=Art   4   det     _   _
2   quick   quick  ADJ    JJ   Degree=Pos                  4   amod    _   _
3   brown   brown  ADJ    JJ   Degree=Pos                  4   amod    _   _
4   fox     fox    NOUN   NN   Number=Sing                 5   nsubj   _   _
5   jumps   jump   VERB   VBZ  Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin   0   root    _   _
6   over    over   ADP    IN   _                           9   case    _   _
7   the     the    DET    DT   Definite=Def|PronType=Art   9   det     _   _
8   lazy    lazy   ADJ    JJ   Degree=Pos                  9   amod    _   _
9   dog     dog    NOUN   NN   Number=Sing                 5   nmod    _   SpaceAfter=No
10  .       .      PUNCT  .    _                           5   punct   _   _


"""

In [21]:
from conllu import parse_tree

AttributeError: 'TokenTree' object has no attribute 'filter'

In [105]:
parse_tree(data)[0].children[0].children

[TokenTree<token={id=1, form=The}, children=None>,
 TokenTree<token={id=2, form=quick}, children=None>,
 TokenTree<token={id=3, form=brown}, children=None>]

In [1]:
from smart_open import open

In [37]:

target_words = {'в':''}
morph_properties = {w: {} for w in target_words}
syntax_properties = {w: {} for w in target_words}


dep_dict = {}
rels = []
for line in open('test.conllu', 'r'): #('/Users/a19336136/rnc_cluster/parsed/corpus1.conllu.gz', "r"):
    print(line)
    if not line.strip():
        continue
    if line.startswith("# "):
        if rels:
            for idx, rel, lemma in rels:
                if idx in dep_dict.keys():
                    rel_set = rel + '|' + '|'.join(sorted(dep_dict[idx]))
                else:
                    rel_set = rel
                if rel_set not in syntax_properties[lemma]:
                    syntax_properties[lemma][rel_set] = 0
                syntax_properties[lemma][rel_set] += 1
        rels = []
        dep_dict = {}        
        continue
    (
        identifier,
        form,
        lemma,
        pos,
        xpos,
        feats,
        head,
        rel,
        enh,
        misc,
    ) = line.strip().split('\t')
    lemma = lemma.lower()
    if head not in dep_dict.keys():
        dep_dict[head] = []
    dep_dict[head].append(rel)

    if lemma in target_words:
        if target_words[lemma]:
            if pos != target_words[lemma]:
                continue
        if feats not in morph_properties[lemma]:
            morph_properties[lemma][feats] = 0
        morph_properties[lemma][feats] += 1
        rels.append((identifier, rel, lemma))

for idx, rel, lemma in rels:  
    rel_set = rel + '|' + '|'.join(sorted(dep_dict[idx]))
    if rel_set not in syntax_properties[lemma]:
            syntax_properties[lemma][rel_set] = 0
    syntax_properties[lemma][rel_set] += 1
rels = []
dep_dict = {}  

# newdoc

# newpar

# sent_id = 1

# text = Материал для скульптора.

1	Материал	МАТЕРИАЛ	NOUN	NN	Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing	0	root	_	_

2	для	ДЛЯ	ADP	IN	_	3	case	_	_

3	скульптора	СКУЛЬПТОР	NOUN	NN	Animacy=Anim|Case=Gen|Gender=Fem|Number=Sing	1	nmod	_	SpaceAfter=No

4	.	.	PUNCT	.	_	1	punct	_	SpacesAfter=\n



# newdoc

# newpar

# sent_id = 1

# text = Виноградов Михаил.

1	Виноградов	ВИНОГРАДОВ	PROPN	NNP	Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing	0	root	_	_

2	Михаил	МИХАИЛ	PROPN	NNP	Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing	1	flat	_	SpaceAfter=No

3	.	.	PUNCT	.	_	1	punct	_	SpacesAfter=\n



# newdoc

# newpar

# sent_id = 1

# text = Материал для скульптора.

1	Материал	МАТЕРИАЛ	NOUN	NN	Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing	0	root	_	_

2	для	ДЛЯ	ADP	IN	_	3	case	_	_

3	скульптора	СКУЛЬПТОР	NOUN	NN	Animacy=Anim|Case=Gen|Gender=Fem|Number=Sing	1	nmod	_	SpaceAfter=No

4	.	.	PUNCT	.	_	1	punct	_	SpacesAfter=\n



# newdoc

# newpar

# sent_id = 1

# text 

In [38]:
syntax_properties

{'в': {'case': 27, 'appos|flat|nmod': 1}}

In [140]:
len(line.split('\t'))

10

In [ ]:
from collections import Counter

In [2]:
da = open('rnc_raw/rnc_raw0.txt.gz').readlines()

In [3]:
from ufal.udpipe import Model, Pipeline

In [4]:
! pip install !wget https://lindat.mff.cuni.cz/repository/xmlui/bitstream/handle/11234/1-3131/russian-syntagrus-ud-2.5-191206.udpipe?sequence=70&isAlloy

zsh:1: no matches found: https://lindat.mff.cuni.cz/repository/xmlui/bitstream/handle/11234/1-3131/russian-syntagrus-ud-2.5-191206.udpipe?sequence=70
zsh:1: command not found: isAlloy


In [5]:
model = Model.load("russian-syntagrus-ud-2.5-191206.udpipe")

In [26]:
with open('test.conllu', 'a') as fw:
    for :
        parsed = pipeline.process(l.strip())
        fw.write(parsed)

In [6]:
pipeline = Pipeline(model, 'generic_tokenizer', '', '', '')
example = "Если бы мне платили каждый раз. Каждый раз, когда я думаю о тебе."
parsed = pipeline.process(da[8].strip())
print(parsed)

# newdoc
# newpar
# sent_id = 1
# text = И вообще,«появилась схема, она работает, решают...
1	И	и	CCONJ	_	_	5	cc	_	_
2	вообще	вообще	ADV	_	Degree=Pos	5	parataxis	_	SpaceAfter=No
3	,	,	PUNCT	_	_	2	punct	_	SpaceAfter=No
4	«	«	PUNCT	_	_	2	punct	_	SpaceAfter=No
5	появилась	появиться	VERB	_	Aspect=Perf|Gender=Fem|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Mid	0	root	_	_
6	схема	схема	NOUN	_	Animacy=Inan|Case=Nom|Gender=Fem|Number=Sing	5	nsubj	_	SpaceAfter=No
7	,	,	PUNCT	_	_	9	punct	_	_
8	она	она	PRON	_	Case=Nom|Gender=Fem|Number=Sing|Person=3	9	nsubj	_	_
9	работает	работать	VERB	_	Aspect=Imp|Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin|Voice=Act	5	conj	_	SpaceAfter=No
10	,	,	PUNCT	_	_	11	punct	_	_
11	решают	решить	VERB	_	Aspect=Imp|Mood=Ind|Number=Plur|Person=3|Tense=Pres|VerbForm=Fin|Voice=Act	5	conj	_	SpaceAfter=No
12	.	.	PUNCT	_	_	11	punct	_	SpaceAfter=No
13	.	.	PUNCT	_	_	5	punct	_	SpaceAfter=No
14	.	.	PUNCT	_	_	5	punct	_	SpacesAfter=\n




In [ ]:
parsed

In [183]:
for i in tqdm(range(7)):
    with open('rnc_raw%s.txt'%(i), 'w') as fw:
        dfAsString = f.RAW[f.shape[0]//7*i:f.shape[0]//7*(i+1)].to_string(header=False, index=False)
        fw.write(dfAsString)        

  0%|          | 0/7 [00:00<?, ?it/s]

In [180]:
for i in range(0, f.shape[0]+1, f.shape[0]//7):
    print(i)

0
3784065
7568130
11352195
15136260
18920325
22704390
26488455


In [177]:
f.shape[0]

26488455